In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [6]:
park_data = pd.read_json('Datasources/other general/parks_datasd.geojson', orient = 'records')

In [7]:
park_data

,features,type
0,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",FeatureCollection
1,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",FeatureCollection
2,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
3,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
4,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
5,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
6,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
7,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
8,"{u'geometry': {u'type': u'Polygon', u'coordina...",FeatureCollection
9,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",FeatureCollection


In [8]:
park_data['features'][272]

{u'geometry': {u'coordinates': [[[[-117.09964458, 32.70127721],
     [-117.09952081, 32.70115748],
     [-117.10016407, 32.70116211],
     [-117.10016147, 32.70061443],
     [-117.10072968, 32.70061844],
     [-117.10075254, 32.70062621],
     [-117.10076872, 32.70063743],
     [-117.10078229, 32.70065433],
     [-117.10078409, 32.70084127],
     [-117.10076406, 32.70098757],
     [-117.10072907, 32.70111017],
     [-117.10067678, 32.7012465],
     [-117.10061846, 32.70131262],
     [-117.10053783, 32.70142874],
     [-117.10037996, 32.70159412],
     [-117.10020963, 32.70171766],
     [-117.10001776, 32.70183115],
     [-117.0999166, 32.7019021],
     [-117.09979699, 32.7020058],
     [-117.0996988, 32.70210618],
     [-117.09963001, 32.70219956],
     [-117.0995626, 32.70229633],
     [-117.09951, 32.7023998],
     [-117.09946272, 32.70249983],
     [-117.09942788, 32.70263944],
     [-117.09941092, 32.70274266],
     [-117.09939933, 32.70293082],
     [-117.09939269, 32.70321525],
 

In [9]:
park_data['type'].unique()

array([u'FeatureCollection'], dtype=object)

In [10]:
df_geo = pd.DataFrame()
count = 0

for idx in park_data.index:
    k = pd.DataFrame.from_dict({count: {'center_x': park_data['features'][idx]['properties']['center_x'], 
                                            'center_y': park_data['features'][idx]['properties']['center_y'],
                                             }}, orient = 'index')
    df_geo = df_geo.append(k)
    
    count = count + 1

In [11]:
df_geo['park_id'] = range(len(df_geo))

In [12]:
df_geo

,center_x,center_y,park_id
0,-117.113,32.708,0
1,-117.161,32.708,1
2,-117.076,33.048,2
3,-117.120,32.970,3
4,-117.130,32.989,4
5,-117.092,32.967,5
6,-117.234,32.948,6
7,-117.133,32.942,7
8,-117.080,32.928,8
9,-117.140,32.914,9


In [13]:
df_geo.to_csv('park_centers.csv')

In [14]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [15]:
schema_two = StructType([StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True)])

In [16]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [17]:
spql_parks_info = sqlContext.createDataFrame(df_geo[['park_id', 'center_x', 'center_y']], schema_two)

In [18]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [19]:
spql_parks_info_rdd = spql_parks_info.rdd.map(lambda row:(row.park_id, row.park_longitude,row.park_latitude))

In [20]:
spql_listings_and_parks_info_rdd = spql_listings_info_rdd.cartesian(spql_parks_info_rdd)

In [21]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [22]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [23]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(distance_calculation)

In [24]:
spql_listings_and_parks_info_rdd.take(2)

[(11204286,
  -117.00194036983295,
  32.581881678016465,
  0,
  -117.113,
  32.708,
  8.729150422837035),
 (11204286,
  -117.00194036983295,
  32.581881678016465,
  1,
  -117.161,
  32.708,
  10.228095266922994)]

In [25]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [26]:
spql_listings_and_parks_info_df = sqlContext.createDataFrame(spql_listings_and_parks_info_rdd, schema_three)

In [29]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.park_id, row.park_longitude, row.park_latitude, row.distance))

In [30]:
close_parks_count = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [31]:
closest_park_distance = spql_listings_and_parks_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [32]:
park_distance  = closest_park_distance.map(lambda x: x[1])

In [33]:
park_distance.take(5)

[0.4226653619251699,
 0.22449796494527174,
 0.40689894777771984,
 0.25248363187723993,
 0.2598624210631214]

In [36]:
spql_listings_and_parks_info_rdd = close_parks_count.zip(park_distance).map(lambda x: (x[0][0], x[0][1], x[1]))

In [37]:
spql_listings_and_parks_info_rdd.take(5)

[(5179392, 263, 0.4226653619251699),
 (7118856, 272, 0.22449796494527174),
 (12953610, 283, 0.40689894777771984),
 (13711374, 283, 0.25248363187723993),
 (13659480, 284, 0.2598624210631214)]

In [41]:
spql_listings_with_parks_info_rdd_row = spql_listings_and_parks_info_rdd.map(lambda x: Row(listing_id = x[0], park_count = x[1], closest_distance = x[2]))

In [42]:
spql_listings_with_parks_info_rdd_row.take(5)

[Row(closest_distance=0.4226653619251699, listing_id=5179392, park_count=263),
 Row(closest_distance=0.22449796494527174, listing_id=7118856, park_count=272),
 Row(closest_distance=0.40689894777771984, listing_id=12953610, park_count=283),
 Row(closest_distance=0.25248363187723993, listing_id=13711374, park_count=283),
 Row(closest_distance=0.2598624210631214, listing_id=13659480, park_count=284)]

In [43]:
sqlContext.createDataFrame(spql_listings_with_parks_info_rdd_row).show(5)

+-------------------+----------+----------+
|   closest_distance|listing_id|park_count|
+-------------------+----------+----------+
| 0.4226653619251699|   5179392|       263|
|0.22449796494527174|   7118856|       272|
|0.40689894777771984|  12953610|       283|
|0.25248363187723993|  13711374|       283|
| 0.2598624210631214|  13659480|       284|
+-------------------+----------+----------+
only showing top 5 rows



In [50]:
listings_parks_information = spql_listings_with_parks_info_rdd_row.toDF().toPandas()

In [56]:
Writer = pd.ExcelWriter('listings_parks_information.xlsx')
listings_parks_information.to_excel(Writer, 'Sheet1')
Writer.save()